In [1]:
from dataset.read_csv import review_data, kapital_val
from transformers import BertForSequenceClassification, logging
from models.sentiment_classifier import SentimentClassifier
from torch.utils.data import DataLoader
from utils import  test_data_tokenizer, label_interpreter


import os
import torch
import tqdm
import random
import numpy as np
import torch.nn as nn
logging.set_verbosity_warning()


/home/mirakram/Projects/sentiment_analysis/sentiment/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13522 entries, 0 to 13521
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               13522 non-null  object 
 1   comment_id       13522 non-null  int64  
 2   datetime         13522 non-null  object 
 3   text             13522 non-null  object 
 4   commenter        13522 non-null  object 
 5   source           13522 non-null  object 
 6   Reported by      13520 non-null  object 
 7   Label            4448 non-null   object 
 8   Sentment         7695 non-null   object 
 9   Label for Jafar  0 non-null      float64
 10  Comment          107 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 1.1+ MB
None
1331 91 0
7 57


<H1> Load the model </h1>

3 models are available for testing.

"Full Azerbaijani" has been trained on nearly 48k data. (remaining 2k data has been lost due unstable internet connection during translation using google translate API)

Other models are trained on 21k data to get the results faster


Time spent for translation algorithm: 24 hours

21k dataset training: 38 mins/epoch


48k dataset training: 1h 18 mins / epoch



In [2]:
available_models = {"Azerbaijani":"models/latest_model_aze.pth",
                    "English": "models/latest_model.pth",
                    "Full Azerbaijani": "models/latest_model_aze_full.pth",
                    "kapital original": "models/kapital_data.pth",
                    "kapital mixed": "models/kapital_data_3_class.pth"}


test_data = review_data("test")
model = SentimentClassifier(2)


print('load model from %s ...' % available_models["English"])
model.load_state_dict(torch.load(available_models["English"])['state_dict'])
print('Done!')


971


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


load model from models/latest_model.pth ...
Done!


<h2> Test single sentence </h2>

In [16]:

from googletrans import Translator
review = "Duzgun istəmir kapital bank az öncə million vasitəsilə balans atkimi etdim, hesaba yuk etməyib digər bankları seçmək lazim İMTİNA. Bu mənim başıma 3-cu dəfədir gəlir bezdirmək"


translator = Translator()
review = translator.translate(review, dest='en', src='az').text
review = "kapital bank does not work properly, i tried to increase my balance, it did not upload. I need to choose another bank. Refuse. this is 3rd time, i experience it"
print(review)
            
model.eval().cpu()

input_ids, attention_masks = test_data_tokenizer(review)
outputs = model(input_ids= input_ids.unsqueeze(dim= 0),
                            attention_mask = attention_masks.unsqueeze(dim = 0))

_, preds = torch.max(outputs, dim=1)

print(preds)

kapital bank does not work properly, i tried to increase my balance, it did not upload. I need to choose another bank. Refuse. this is 3rd time, i experience it
tensor([0])


<h2> Batch Testing with test data </h2>

In [4]:
test_data = kapital_val("train")

dataloader_test = DataLoader(
    test_data,
    batch_size=1
)

print("if cuda is working 1 min. if not around 10 mins)")
tq = tqdm.tqdm(total = len(dataloader_test))
tq.set_description('Test')

test_iterator = enumerate(dataloader_test)
correct_predictions=0
model.eval().cuda()

for _, batch in test_iterator:
    with torch.no_grad():
        input_ids, attention_masks, label = batch

        outputs = model(input_ids= input_ids.cuda(),
                                attention_mask = attention_masks.cuda())
                
        _, preds = torch.max(outputs, dim=1)
        correct_predictions += torch.sum(preds == label.cuda())
                
        tq.update()

tq.close()

print("Accuracy: ", correct_predictions/len(dataloader_test))



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13522 entries, 0 to 13521
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               13522 non-null  object 
 1   comment_id       13522 non-null  int64  
 2   datetime         13522 non-null  object 
 3   text             13522 non-null  object 
 4   commenter        13522 non-null  object 
 5   source           13522 non-null  object 
 6   Reported by      13520 non-null  object 
 7   Label            4448 non-null   object 
 8   Sentment         7695 non-null   object 
 9   Label for Jafar  0 non-null      float64
 10  Comment          107 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 1.1+ MB
None
1331 91 0
79 1323
if cuda is working 1 min. if not around 10 mins)


Test: 100%|██████████| 1323/1323 [00:34<00:00, 38.05it/s]

Accuracy:  tensor(0.0907, device='cuda:0')
